In [ ]:
# %load ../_data/standard_import.txt

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-white')

## geoPy

[read the docs](http://geopy.readthedocs.io/en/stable/)

In [ ]:
# !pip install geopy

#### Nominatim - Open Street Map

https://wiki.openstreetmap.org/wiki/Nominatim

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("9, Singel, Amsterdam")
location.raw
location.address
(location.latitude, location.longitude)


In [ ]:
location = geolocator.reverse("52.25, 4.99")

In [ ]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(format_string="%s, Netherlands")
address, (latitude, longitude) = geolocator.geocode("amsterdam")
address, latitude, longitude

In [ ]:
from geopy.geocoders import GeoNames
geolocator = GeoNames(username='user_name_here', format_string="%s")
geolocator.geocode("amsterdam")

In [ ]:
current_ip_location = !curl ipinfo.io/loc
latlon = current_ip_location[-1].split(',')
geolocator.reverse('{}, {}'.format(latlon[0], latlon[1]))

#### IP info

In [ ]:
ip_location = !curl ipinfo.io
ip_location

### GeoPy - geolocator

In [ ]:
def ip_location():
    """Returns latlon of IP location of provider."""
    current_ip_location = !curl ipinfo.io/loc
    latlon = current_ip_location[-1].split(',')
    return geolocator.reverse('{}, {}'.format(latlon[0], latlon[1])), latlon[0], latlon[1], 0.
ip_location()

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("9, Singel, Amsterdam")
location.raw
location.address
(location.latitude, location.longitude)


## Here

In [ ]:
import json
import os
import platform
import re
import ssl
import subprocess
import requests
import urllib.request
from urllib.request import urlretrieve
import plistlib

from PIL import Image

#### Credentials

In [ ]:
import os, pickle

pickle_file='../_credentials/here_credentials.pkl'
if not os.path.exists(pickle_file):
    Here={}
    Here['app_id'] = 'wL...9'
    Here['app_code'] = '9..._-A'
    with open(pickle_file, 'wb') as f:
        pickle.dump(Here, f)
else:
    Here=pickle.load(open(pickle_file,'rb'))

#### URL and headers for HERE location api

In [ ]:
context = ssl._create_unverified_context()
positioning_headers = {'Content-Type': 'application/json'}
positioning_url = 'https://pos.api.here.com/positioning/v1/locate?app_id={}&app_code={}'.format(Here['app_id'], Here['app_code'])
rev_geocoder_url = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json?prox='

### Search address

In [ ]:
def geo_address(api_url, searchtext):
    """Lookup address details as JSON
    Decode UTF-8 bytes to Unicode and convert single quotes to double quotes to make it valid JSON.
    Load the JSON to a Python list & dump it back out as formatted JSON."""
    
    http = '{}?app_id={}&app_code={}&searchtext={}'.format(api_url, Here['app_id'], Here['app_code'], searchtext)
    content = requests.get(http).content
    my_json = content.decode('utf8').replace("'", '"')
    data = json.loads(my_json)
    
    return data

In [ ]:
def recursive_keys(json, values=False):
    """Recursively get keys and list items"""
    if type(json) == dict:
        for key, value in json.items():
            if values==False: value = ''
            if type(json[key]) == str: 
                print('--', key, ':',value)
                continue
            elif type(json[key]) == list: 
                print('list', key, ':', value)
                recursive_keys(json[key]) # [0]
            elif type(json[key]) == dict: 
                print('dict', key, ':', value)
                recursive_keys(json[key])

    elif type(json) == list:
        for i, item in enumerate(json):
            print(i, end=' ')
            recursive_keys(json[i])

In [ ]:
api_url_places = 'https://geocoder.cit.api.here.com/6.2/geocode.json'
geo_bb = geo_address(api_url_places, '9, Singel, Amsterdam')
geo_bb

In [ ]:
geo_address(api_url_places, '9, Singel, Amsterdam')['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]

In [ ]:
recursive_keys(geo_bb)

In [ ]:
def geo_location(api_url, payload):
    """Lookup address details as JSON
    Decode UTF-8 bytes to Unicode and convert single quotes to double quotes to make it valid JSON.
    Load the JSON to a Python list & dump it back out as formatted JSON."""

    http = '{}?app_id={}&app_code={}'.format(api_url, Here['app_id'], Here['app_code'])
    headers = {'Content-Type': 'application/json'}
    r = requests.post(http, data=json.dumps(payload), headers=headers)
    print(r.status_code, r.reason)
    my_json = r.content.decode('utf8').replace("'", '"')
    
    return json.loads(my_json)

In [ ]:
api_url_locate = 'https://pos.cit.api.here.com/positioning/v1/locate'
payload = {"wlan":[
    {"mac":"6a:d8:2f:0e:69:17", "powrx":-70},
    {"mac":"38:d8:2f:0e:69:16", "powrx":-72},
    {"mac":"80:ea:96:eb:90:47", "powrx":-61},
    {"mac":"48:ee:0c:f1:27:d9", "powrx":-71},
    {"mac":"80:ea:96:eb:90:46", "powrx":-55}]}

loc_bb = geo_location(api_url_locate, payload)
loc_bb

In [ ]:
def rev_geocoder(lat, lon, radius='N/A'):
    results = json.load(urllib.request.urlopen(
        '{}{},{},1000&mode=retrieveAddresses&maxResults=3&gen=8&app_id={}&app_code={}'.format(
            rev_geocoder_url, lat, lon, Here['app_id'], Here['app_code']), context=context)
                       )['Response']['View'][0]['Result']
    for result in results:
        if result['MatchQuality'].get('Street'):
            address = result['Location']['Address']['Label']
            break
        else:
            continue
    print('\nHERE location: {}, (lat: {},lon: {}, radius: {})'.format(address, lat, lon, radius))

    return address

In [ ]:
def mia_saved_map(lon, lat, radius):
    mia_url = 'http://image.maps.api.here.com/mia/1.6/mapview?c={},{}&u={}&w=1280&h=1024' \
              '&app_id={}&app_code={}&t=3'.format(lat, lon, radius, Here['app_id'], Here['app_code'])
    picFileName = "miaPic.jpg"
    urlretrieve(mia_url, picFileName)
    Image.open(picFileName).show()

In [ ]:
def xml2dict(xml):
    """Convert XML to dictionary format."""
    import plistlib
    return plistlib.loads(str.encode(xml), fmt=plistlib.FMT_XML)

In [ ]:
def format_bssid(bssid):
    """Format BSSID when needed."""
    if not re.findall('\w\w:\w\w:\w\w:\w\w:\w\w:\w\w', bssid):
        bssid_arr = bssid.split(':')
        bssid_modified_arr = []
        for bssid_elem in bssid_arr:
            if len(bssid_elem) == 1:
                bssid_elem = '0{}'.format(bssid_elem)
            bssid_modified_arr.append(bssid_elem)
        bssid = ':'.join(bssid_modified_arr)
    return bssid

In [ ]:
def wifi_addresses(printout=True):
    """Return payload for POST request into HERE api."""
    
    # Library to convert literal dict to type dict
    import ast
    
    try:
        mac_list = set()    
        # Windows
        if platform.system() == 'Windows':
            results = subprocess.check_output(["netsh", "wlan", "show", "network", "bssid"])
            results = results.decode("ascii")  # needed in python 3
            list = results.replace('\r', '').split('\n')
            for element in list:
                if re.match('.*BSSID.*', element):
                    mac_list.add('{"mac": "' + (element.split(' : ')[1]) + '"}')
        # macOS           
        elif platform.system() == 'Darwin':

            # Save WIFI list in XML format
            wifi_list = os.popen('airport -s -x').read()
            open('wifi_plist.txt', 'w').write(wifi_list)

            # Convert XML to JSON/dict format
            hotspots_plist = xml2dict(wifi_list)

            for hotspot in hotspots_plist:
                ssid_str = hotspot['SSID_STR']
                rssi = hotspot['RSSI']
                bssid = format_bssid(hotspot['BSSID'])
                print('SSID: {}, channel: {}, Mac:{}, signal: {}'.format(ssid_str, hotspot['CHANNEL'], bssid, rssi))

                mac_list.add('{\"mac\":\"' + bssid + '\", \"powrx\":' + str(rssi) + '}')  # BSSID + Powrx
    except:
        return None
    
    payload = '\n{"wlan":[\n' + ',\n'.join(i for i in mac_list) + ']}'
    if printout: print('\nPOST request body:\n{}'.format(payload))
    
    return ast.literal_eval(payload)

In [ ]:
def wifi_positioning(saved_map=False):
    """Return current location based on nearby wifi-hotspots."""
    
    payload = wifi_addresses()
    
    if payload is None:
        print('WIFI positioning failed, returning IP location from provider')
        _, lat, lon, radius = ip_location()
        latlon = '{}, {}'.format(lat, lon)
    else:
        api_url_locate = 'https://pos.cit.api.here.com/positioning/v1/locate'
        json_result = geo_location(api_url_locate, payload)
        lat, lon = json_result['location']['lat'], json_result['location']['lng']
        latlon = '{}, {}'.format(lat, lon)
        radius = json_result['location']['accuracy']

    # Print address and lat, lon, radius and open/show location on map
    print('\ngeoPy Nominatim location: ', geolocator.reverse(latlon))
    rev_geocoder(lat, lon, radius)
    here_maps = 'https://wego.here.com/directions/mix/{},{}/?map={},{},15'.format(lat, lon, lat, lon)
    google_maps = 'https://www.google.com/maps/search/{},{}/@{},{},17z'.format(lat, lon, lat, lon)
    !open -a Safari $here_maps; open -a Safari $google_maps
    if saved_map: mia_saved_map(lon, lat, radius)
        
    return lat, lon

In [ ]:
lat, lon = wifi_positioning()